In [1]:
import pandas as pd 
import numpy as np

In [2]:

data=pd.read_csv(r"C:\Users\CG-DTE\Desktop\AI\GITHUB\Predicting bankruptcy\low level model analysis\4year.csv")

C:\Users\CG-DTE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,6,7,8,10,11,14,15,16,17,18,22,24,25,26,29,34,35,36,38,48,50,51,57,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### above feature set i have gone through some feature engineering steps
#### missing imputation------>gaussian transformation ----->decision tree descritisation------>scaling --->feature selection
#### [['Attr4','Attr5', 'Attr9', 'Attr13','Attr15', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr34', 'Attr35', 'Attr40','Attr44' 'Attr46', 'Attr56']] i got these features

### but important analysis i found that feature set using these above feature engineering steps giving best result on original data aftter imputation only

In [3]:
year_4=data[['Attr5','Attr9','Attr15','Attr21','Attr24','Attr27','Attr34','Attr35','Attr40','Attr46','Attr55','Attr58','class']]

In [4]:
k1=year_4.columns.tolist()

In [5]:
#replacing each ? to NAN
for i in k1:
    year_4[i] = year_4[i].replace('?' , np.nan)

<ipython-input-5-763fa34d3bd1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_4[i] = year_4[i].replace('?' , np.nan)


In [6]:
#converting object data into numericals(float64)
year_4=year_4.apply(lambda col:pd.to_numeric(col,errors='coerce'))

In [7]:
year_4

,Attr5,Attr9,Attr15,Attr21,Attr24,Attr27,Attr34,Attr35,Attr40,Attr46,Attr55,Attr58,class
0,-44.8530,1.12230,771.49,1.03460,0.576070,1.44230,0.391040,0.180820,0.167020,0.77736,90533.00,0.89101,0
1,7.5970,1.29440,-1871.30,0.65665,NaN,-18.99600,3.049300,-0.115680,0.042359,1.08630,2624.90,1.06250,0
2,125.6800,1.05740,725.64,0.99173,0.208350,0.71641,0.325060,0.076618,1.145100,2.56420,24672.00,0.94571,0
3,19.1150,1.11440,924.13,1.33550,0.504970,1.07750,0.357240,0.144820,0.218530,1.19710,6649.90,0.89737,0
4,-15.3440,1.73500,1663.00,1.03080,NaN,0.00000,1.392400,0.050620,0.245800,1.13870,1314.00,0.57153,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9787,-39.0640,0.97093,2892.60,0.97345,0.013002,NaN,0.052162,0.028662,0.005245,0.73857,8772.00,1.02990,1
9788,-20.9230,1.00730,4654.80,0.76438,-0.027610,NaN,0.004403,0.002675,0.115010,0.47808,-564.42,0.99280,1
9789,-1.0692,0.80307,-1267.00,0.38471,-0.240360,NaN,-0.379840,-0.238190,0.004595,0.98745,7371.80,1.24520,1
9790,-214.2400,0.98145,2635.80,0.95615,-0.018774,3.07160,1.239600,0.090025,0.002242,0.39113,-4346.00,0.68127,1


### so here i m building model on entire dataset for , so i m not divinding into train and test dataset , so that i do have ample amount of data to train our model more accurately

In [8]:
predictors=['Attr5','Attr9','Attr15','Attr21','Attr24','Attr27','Attr34','Attr35','Attr40','Attr46','Attr55','Attr58']
target=['class']
X=year_4[predictors]
y=year_4[target]

In [9]:
#X
#y

In [10]:
from sklearn.impute import SimpleImputer

In [11]:
# Now we impute the missing values with SimpleImputer

# create an instance of the simple imputer
# we indicate that we want to impute with the median
mean_imputer = SimpleImputer(strategy='mean',)

# we fit and transform the predictors dataframe
# the imputer will learn the mean of all variables
X_new=mean_imputer.fit_transform(X[predictors])

In [12]:
y_1=y.copy()
X_1=pd.DataFrame(X_new, columns=predictors)

### over sampling through SMOTE is performed

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2) 
X_res, y_res = sm.fit_resample(X_1, y_1)

In [14]:
len(X_res)

18554

In [15]:

len(y_res)

18554

### Note:- since , i have not splited my data into train-test dataset so my independent and dependent data are in pandas dataframe so in order to implement in XGBoostingClassifier algo in pickled , model will not take dataframe data as input to predict , so i need to convert X(input data ) into numpy array , i m not converting y into numpy array¶


In [16]:
#type(X_res)
type(y_res)

pandas.core.frame.DataFrame

In [17]:
X=np.array(X_res)
y=np.array(y_res)

In [18]:
#type(X)
#type(y)

In [19]:
import seaborn as sns
import xgboost as xgb
# Classification metrices
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_score,recall_score,f1_score

In [20]:
xgb.__version__

'1.5.0'

### functionalizing the model

In [21]:

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import pickle

In [22]:
# Functionalize model fittting

def FitModel(X,Y,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2)
    
    
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    
    
    grid_result = grid.fit(x_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(x_test)
    cm = confusion_matrix(y_test, pred)
   # metrics =grid_result.gr
    print(pred)
    pickle.dump(grid_result,open(algo_name,'wb'))
   
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n', cm)

In [23]:
param ={
            'n_estimators': [50, 100, 500, 1000, 2000],
           
        }
FitModel(X,y,'finalized_model2.pkl',XGBClassifier(),param,cv=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\CG-DTE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\CG-DTE\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\CG-DTE\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[18:42:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[1 1 0 ... 0 1 1]
Best Params : {'n_estimators': 2000}
Classification Report :               precision    recall  f1-score   support

           0       0.99      0.97      0.98      1838
           1       0.97      0.99      0.98      1873

    accuracy                           0.98      3711
   macro avg       0.98      0.98      0.98      3711
weighted avg       0.98      0.98      0.98      3711

Accuracy Score : 0.9770951226084613
Confusion Matrix : 
 [[1777   61]
 [  24 1849]]


### Reloading the Saved Model

In [24]:
loaded_model = pickle.load(open("finalized_model2.pkl","rb"))

In [25]:
pred = loaded_model.predict(np.array([[-214.2400,0.98145,2635.80,0.95615,-0.018774,3.0716,1.239600,0.090025,0.002242,0.39113,-4346.00,0.68127]]))

In [26]:

print(pred)

[1]


In [27]:
def change (prediction):
    if prediction==1:
        return 'yes'
    else:
        return 'No'

In [28]:
print('some of the features of company:-')
print('[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365 = -214.2400 ')
print('sales / total assets = 0.98145')
print('(total liabilities * 365) / (gross profit + depreciation) = 2635.80')
print('sales (n) / sales (n-1) = 0.95615')
print('gross profit (in 3 years) / total assets = -0.018774')
print('profit on operating activities / financial expenses = 3.0716')
print('operating expenses / total liabilities = 1.239600')
print('profit on sales / total assets = .090025')
print('(current assets - inventory - receivables) / short-term liabilities = 0.002242')
print('(current assets - inventory) / short-term liabilities = 0.39113')
print('working capital = -4346.00')
print('total costs /total sales = 0.68127')

print('\n\n')

print('will company go for bankruptcy ? {}'.format(change(pred)))

some of the features of company:-
[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365 = -214.2400 
sales / total assets = 0.98145
(total liabilities * 365) / (gross profit + depreciation) = 2635.80
sales (n) / sales (n-1) = 0.95615
gross profit (in 3 years) / total assets = -0.018774
profit on operating activities / financial expenses = 3.0716
operating expenses / total liabilities = 1.239600
profit on sales / total assets = .090025
(current assets - inventory - receivables) / short-term liabilities = 0.002242
(current assets - inventory) / short-term liabilities = 0.39113
working capital = -4346.00
total costs /total sales = 0.68127



will company go for bankruptcy ? yes
